In [54]:
import boto3
import os


class Bucket:
    def __init__(self, region, namespace, bucket_name):
        self.region = region,
        self.namespace = namespace,
        self.bucket_name = bucket_name

        OCI_ACCESS_KEY_ID = os.environ['OCI_ACCESS_KEY_ID']
        OCI_SECRET_ACCESS_KEY = os.environ['OCI_SECRET_ACCESS_KEY']
        OCI_REGION = region
        OCI_NAMESPACE = namespace

        session = boto3.session.Session()

        self.s3_client = session.client(
            's3',
            region_name=OCI_REGION,
            endpoint_url=f'https://{OCI_NAMESPACE}.compat.objectstorage.{OCI_REGION}.oraclecloud.com',
            aws_access_key_id=OCI_ACCESS_KEY_ID,
            aws_secret_access_key=OCI_SECRET_ACCESS_KEY
        )
    
    def delete_folder_content(self, bucket_folder):
        folder_prefix = bucket_folder

        delete_objects = []

        while True:
            response = self.s3_client.list_objects_v2(Bucket=self.bucket_name, Prefix=folder_prefix)
            
            if 'Contents' in response:
                delete_objects.extend([{'Key': obj['Key']} for obj in response['Contents']])
                
                # Delete in batches of 1000
                while len(delete_objects) >= 1000:
                    self.s3_client.delete_objects(Bucket=self.bucket_name, Delete={'Objects': delete_objects[:1000]})
                    delete_objects = delete_objects[1000:]
                
                # Check if there's more data to paginate
                if response.get('IsTruncated'):
                    continuation_token = response.get('NextContinuationToken')
                    response = self.s3_client.list_objects_v2(Bucket=self.bucket_name, Prefix=folder_prefix, ContinuationToken=continuation_token)
                else:
                    break
            else:
                break

        # Delete remaining objects (less than 1000)
        if delete_objects:
            self.s3_client.delete_objects(Bucket=self.bucket_name, Delete={'Objects': delete_objects})

        print(f"All objects in the folder '{folder_prefix}' have been deleted.")


    def list_objects(self):
        response = self.s3_client.list_objects_v2(Bucket=self.bucket_name)
        try:
            for obj in response["Contents"]:
                print(obj['Key'])
        except KeyError as e:
            print("No content to list.")

    def put_object(self, 
                   local_folder, 
                   local_filename, 
                   destination_folder,
                   destination_filename):
        """
        local_folder 
        local_filename 
        destination_folder
        destination_filename
        """
        with open(f"{local_folder}{local_filename}", "rb") as file:
            self.s3_client.put_object(Body=file, 
                                      Key=f"{destination_folder}{destination_filename}",
                                      Bucket=self.bucket_name)
            
        
    def close(self):
        self.s3_client.close()

In [45]:
OCI_REGION ='uk-london-1'
OCI_NAMESPACE = 'lrqgbz9z6zlj'
bucket_name = 'london-property-sales-price'

bucket = Bucket(OCI_REGION, OCI_NAMESPACE, bucket_name)

In [59]:
bucket.list_objects()

No content to list.


In [58]:
bucket.delete_folder_content("ciaofolder")

All objects in the folder 'ciaofolder' have been deleted.


In [56]:
bucket.put_object(local_folder = 'prova/',
                  local_filename = 'provafile.txt',
                  destination_folder = 'ciaofolder/',
                  destination_filename = 'provafile.txt'
)

In [60]:
bucket.close()

In [22]:
def clear_buckets(region,
                  namespace, 
                  bucket_name,
                  bucket_folder_list):

    bucket = Bucket(region, namespace, bucket_name)
    
    for folder in bucket_folder_list:
        bucket.delete_folder_content(folder)

    bucket.close()

In [24]:
bucket_folder_list = ["ppd-download-chunks/", "epc-download-chunks/"]

clear_buckets(OCI_REGION, OCI_NAMESPACE, bucket_name, bucket_folder_list)

All objects in the folder 'ppd-download-chunks/' have been deleted.
All objects in the folder 'epc-download-chunks/' have been deleted.


In [61]:
!rm -rf prova/

In [31]:
OCI_ACCESS_KEY_ID = os.environ['OCI_ACCESS_KEY_ID']
OCI_SECRET_ACCESS_KEY = os.environ['OCI_SECRET_ACCESS_KEY']
OCI_REGION = 'uk-london-1'
OCI_NAMESPACE = 'lrqgbz9z6zlj'
bucket_name = 'london-property-sales-price'


session = boto3.session.Session()

s3_client = session.client(
    's3',
    region_name=OCI_REGION,
    endpoint_url=f'https://{OCI_NAMESPACE}.compat.objectstorage.{OCI_REGION}.oraclecloud.com',
    aws_access_key_id=OCI_ACCESS_KEY_ID,
    aws_secret_access_key=OCI_SECRET_ACCESS_KEY
)

In [41]:
response = s3_client.list_objects_v2(Bucket=bucket_name)
try:
    for obj in response["Contents"]:
        print(obj['Key'])
except KeyError as e:
    print("No content to list.")

prova/provafile.txt
prova1/provafile.txt
prova2/provafile.txt


In [42]:
# with open("prova/provafile.txt", "rb") as file:
#     s3_client.put_object(Body=file,Key="prova3/provafile.txt", Bucket=bucket_name)

In [43]:
s3_client.close()